# 属性

在`Python`中，数据的属性和处理数据的方法统称属性`attribute`.其实，方法只是可调用的属性.

Python提供了丰富的API，用于控制属性的访问权限，以及实现动态属性.

使用点号访问属性时(如`obj.attr`)，Python解释器会调用特殊的方法(如`__getattr__`和`__setattr__`)计算属性.用户自己定义的类可以通过`__getattr__`方法实现“虚拟属性”.当访问不存在的属性时（如`obj.no_such_attribute`），即时计算属性的值.

动态创建属性照理说是一种元编程，框架的作者经常这么做.然而，在Python中，相关的基础技术十分简单，任何人都可以使用，甚至在日常的数据转换任务中也能用到.下面以这种任务开启本章的话题.

## 动态属性

python提供了处理动态属性相关的api

### 影响属性处理方式的特殊属性


+ `__class__`

    对象所属类的引用（即`obj.__class__` 与`type(obj)` 的作用相同）.Python 的某些特殊方法，例如`__getattr__`，只在对象的类中寻找，而不在实例中寻找.

+ `__dict__`

    一个映射，存储对象或类的可写属性.有`__dict__` 属性的对象，任何时候都能随意设置新属性。如果类有`__slots__`属性，它的实例可能没有`__dict__`属性。参见下面对`__slots__` 属性的说明.

+ `__slots__`

    类可以定义这个这属性，限制实例能有哪些属性.`__slots__`属性的值是一个字符串组成的元组，指明允许有的属性.如果`__slots__` 中没有`'__dict__'`，那么该类的实例没有`__dict__` 属性，实例只允许有指定名称的属性。

### 处理属性的内置函数

下述5 个内置函数对对象的属性做读、写和内省操作.

+ dir([object])

    列出对象的大多数属性.dir 函数的目的是交互式使用，因此没有提供完整的属性列表，只列出一组“重要的”属性名。`dir` 函数能审查有或没有`__dict__`属性的对象.`dir`函数不会列出`__dict__`属性本身，但会列出其中的键.`dir` 函数也不会列出类的几个特殊属性，例如`__mro__`、`__bases__` 和`__name__`.如果没有指定可选的`object` 参数，`dir` 函数会列出当前作用域中的名称.

+ getattr(object, name[, default])

    从`object` 对象中获取name字符串对应的属性.获取的属性可能来自对象所属的类或超类。如果没有指定的属性，`getattr` 函数抛出`AttributeError`异常，或者返回`default`参数的值(如果设定了这个参数的话).

+ hasattr(object, name)

    如果`object`对象中存在指定的属性，或者能以某种方式(例如继承)通过`object`对象获取指定的属性， 返回True
    
+ setattr(object, name, value)

    把`object`对象指定属性的值设为`value`，前提是`object` 对象能接受那个值.这个函数可能会创建一个新属性，或者覆盖现有的属性.


+ vars([object])

    返回`object`对象的`__dict__`属性；如果实例所属的类定义了`__slots__` 属性，实例没有`__dict__` 属性，那么`vars` 函数不能处理那个实例(相反，`dir` 函数能处理这样的实例).如果没有指定参数，那么`vars()`函数的作用与`locals()`函数一样：返回表示本地作用域的字典.
    
  
  
### 处理属性的特殊方法

在用户自己定义的类中，下述特殊方法用于获取、设置、删除和列出属性.

使用点号或内置的`getattr`、`hasattr` 和`setattr` 函数存取属性都会触发下述列表中相应的特殊方法.但是，直接通过实例的`__dict__`属性读写属性不会触发这些特殊方法——如果需要，通常会使用这种方式跳过特殊方法.


对用户自己定义的类来说，如果隐式调用特殊方法，仅当特殊方法在对象所属的类型上定义，而不是在对象的实例字典中定义时，才能确保调用成功.


要假定特殊方法从类上获取，即便操作目标是实例也是如此。因此，特殊方法不会被同名实例属性遮盖。


+ `__delattr__(self, name)`

    只要使用`del`语句删除属性，就会调用这个方法。例如，`del obj.attr` 语句触发`Class.__delattr__(obj, 'attr')`方法.

+ `__dir__(self)`

    把对象传给dir 函数时调用，列出属性。例如，`dir(obj)` 触发`Class.__dir__(obj)`方法.

+ `__getattr__(self, name)`

    仅当获取指定的属性失败，搜索过`obj`、`Class`和超类之后调用.表达式`obj.no_such_attr`、`getattr(obj, 'no_such_attr')` 和`hasattr(obj, 'no_such_attr')`可能会触发`Class.__getattr__(obj, 'no_such_attr')` 方法，但是，仅当在`obj`、`Class` 和超类中找不到指定的属性时才会触发.
    
+ `__getattribute__(self, name)`

    尝试获取指定的属性时总会调用这个方法，不过，寻找的属性是特殊属性或特殊方法时除外。点号与`getattr` 和`hasattr` 内置函数会触发这个方法.调用`__getattribute__`方法且抛出`AttributeError` 异常时， 才会调用`__getattr__` 方法。为了在获取`obj`实例的属性时不导致无限递归，`__getattribute__` 方法的实现要使用`super().__getattribute__(obj, name)`。

+ `__setattr__(self, name, value)`

    尝试设置指定的属性时总会调用这个方法.点号和`setattr`内置函数会触发这个方法.例如，`obj.attr = 42` 和`setattr(obj, 'attr', 42)` 都会触发`Class.__setattr__(obj,attr’, 42)` 方法。


其实，特殊方法`__getattribute__` 和`__setattr__` 不管怎样都会调用，几乎会影响每一次属性存取，因此比`__getattr__` 方法（只处理不存在的属性名）更难正确使用.与定义这些特殊方法相比，使用特性或描述符相对不易出错。

### 例子

我们要使用动态属性处理O’Reilly 为OSCON 2014 大会提供的JSON 格式数据源.

那个JSON源中有895条记录，整个数据集是一个JSON 对象，里面有一个键，名为"Schedule"；这个键对应的值也是一个映像，有4 个键："conferences"、"events"、"speakers" 和"venues"。这4 个键对应的值都是一个记录列表。
列表中有成百上千条记录。不过，"conferences" 键对应的列表中只有一条记录，如上述示例所示。这4 个列表中的每个元素都有一个名为"serial" 的字段，这是元素在各个列表中的唯一标识符.

第一个脚本只用于下载那个OSCON 数据源.为了避免浪费流量，我会先检查本地有没有副本。这么做是合理的，因为OSCON 2014 大会已经结束，数据源不会再更新。
第一个例子没用到元编程，几乎所有代码的作用可以用这一个表达式概括：`json.load(fp)`.
不过，这样足以处理那个数据集了.osconfeed.load 函数会在后面几个示例中用到.

In [1]:
import requests
import warnings
import os
import json
URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'osconfeed.json'
def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg) 
        with open(JSON, 'w') as local:
            remote = requests.get(URL)
            json.dump(remote.json(),local)
    with open(JSON) as fp:
        return json.load(fp)

In [2]:
raw_feed = load()

In [3]:
sorted(raw_feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [4]:
for key, value in sorted(raw_feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key))

  1 conferences
494 events
357 speakers
 53 venues


In [5]:
raw_feed['Schedule']['speakers'][-1]['name']

'Carina C. Zona'

In [6]:
raw_feed['Schedule']['speakers'][-1]['serial']

141590

In [7]:
raw_feed['Schedule']['events'][40]['name']

'There *Will* Be Bugs'

In [8]:
raw_feed['Schedule']['events'][40]['speakers']

[3471, 5199]

### 使用动态属性访问JSON类数据

`feed['Schedule']['events'][40]['name']` 这种句法很冗长.在JavaScript中，可以使用`feed.Schedule.events[40].name `获取那个值。在Python 中，可以实现一个近似字典的类（网上有大量实现），达到同样的效果。我自己实现了`FrozenJSON`类，比大多数实现都简单，因为只支持读取，即只能访问数据。不过，这个类能递归，自动处理嵌套的映射和列表


In [9]:
from collections import abc
class FrozenJSON:
    """一个只读接口，使用属性表示法访问JSON类对象
    """
    def __init__(self, mapping):
        self.__data = dict(mapping) 
    def __getattr__(self, name): 
        # `__getattr__`特殊方法用于重载`.`符号获取值的行为
        if hasattr(self.__data, name):
            return getattr(self.__data, name) 
        else:
            return FrozenJSON.build(self.__data[name])
        
    @classmethod
    def build(cls, obj): 
        if isinstance(obj, abc.Mapping): 
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence): 
            return [cls.build(item) for item in obj]
        else: 
            return obj

In [10]:
feed = FrozenJSON(raw_feed)

In [11]:
len(feed.Schedule.speakers)

357

In [12]:
sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [13]:
feed.Schedule.speakers[-1].name

'Carina C. Zona'

In [14]:
talk = feed.Schedule.events[40]

In [15]:
type(talk)

__main__.FrozenJSON

In [16]:
talk.name

'There *Will* Be Bugs'

In [17]:
talk.speakers

[3471, 5199]

In [18]:
talk.flavor

KeyError: 'flavor'

### 处理无效属性名

FrozenJSON 类有个缺陷：没有对名称为Python 关键字的属性做特殊处理。比如说像下面这
样构建一个对象

In [19]:
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})

此时无法读取`grad.class`的值，因为在Python中class是保留字:

In [20]:
grad.class

SyntaxError: invalid syntax (<ipython-input-20-ce4327ea3f6c>, line 1)

但是，FrozenJSON 类的目的是为了便于访问数据，因此更好的方法是检查传给`Frozen-JSON.__init__` 方法的映射中是否有键的名称为关键字，如果有，那么在键名后加上`_`.

这种有问题的键在Python 3中易于检测，因为str类提供的`s.isidentifier()`方法能根据语言的语法判断`s`是否为有效的Python标识符.但是，把无效的标识符变成有效的属性名却不容易.对此，有两个简单的解决方法:

+ 一个是抛出异常
+ 另一个是把无效的键换成通用名称，例如`attr_0`、`attr_1`，等等.

为了简单起见，我将忽略这个问题.

对动态属性的名称做了一些处理之后，我们要分析`FrozenJSON`类的另一个重要功能——类方法`build`的逻辑。这个方法把嵌套结构转换成`FrozenJSON`实例或`FrozenJSON`实例列表，因此`__getattr__` 方法使用这个方法访问属性时，能为不同的值返回不同类型的对象.

In [21]:
import keyword
from collections import abc
class FrozenJSON:
    """一个只读接口，使用属性表示法访问JSON类对象
    """
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value
            
    def __getattr__(self, name): 
        # `__getattr__`特殊方法用于重载`.`符号获取值的行为
        if hasattr(self.__data, name):
            return getattr(self.__data, name) 
        else:
            return FrozenJSON.build(self.__data[name])
        
    @classmethod
    def build(cls, obj): 
        if isinstance(obj, abc.Mapping): 
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence): 
            return [cls.build(item) for item in obj]
        else: 
            return obj

In [22]:
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})

In [23]:
grad.class_

1982

### 使用`__new__`方法以灵活的方式创建对象

除了在类方法中实现这样的逻辑之外，还可以在特殊的`__new__`方法中实现.

我们通常把`__init__`称为构造方法，这是从其他语言借鉴过来的术语.其实，用于构建实例的是特殊方法`__new__`--这是个类方法(使用特殊方式处理，因此不必使用`@classmethod`装饰器)，必须返回一个实例。返回的实例会作为第一个参数（即`self`）传给`__init__`方法。因为调用`__init__`方法时要传入实例，而且禁止返回任何值，所以`__init__` 方法其实是“初始化方法”.真正的构造方法是`__new__`。我们几乎不需要自己编写`__new__`方法，因为从`object`类继承的实现已经足够了。

刚才说明的过程，即从`__new__ `方法到`__init__ `方法，是最常见的，但不是唯一的.

`__new__`方法也可以返回其他类的实例，此时，解释器不会调用`__init__`方法.

下面是FrozenJSON 类的另一个版本，把之前在类方法build 中的逻辑移到了`__new__`方法中.

In [24]:
import keyword
from collections import abc
class FrozenJSON:
    """一个只读接口，使用属性表示法访问JSON类对象
    """
    def __new__(cls, arg): 
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)
        elif isinstance(arg, abc.MutableSequence): 
            return [cls(item) for item in arg]
        else:
            return arg
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value
            
    def __getattr__(self, name): 
        # `__getattr__`特殊方法用于重载`.`符号获取值的行为
        if hasattr(self.__data, name):
            return getattr(self.__data, name) 
        else:
            return FrozenJSON.build(self.__data[name])
        

`__new__`方法的第一个参数是类,因为创建的对象通常是那个类的实例.所以,在`FrozenJSON.__new__`方法中,`super().__new__(cls)`表达式会调`object.__new__(FrozenJSON)`，
而`object`类构建的实例其实是`FrozenJSON`实例，即那个实例的`__class__`属性存储的是
`FrozenJSON`类的引用.不过，真正的构建操作由解释器调用`C`语言实现的`object.__new__`方法执行.

OSCON的JSON数据源有一个明显的缺点：索引为40的事件，即名为`There *Will* Be Bugs`的那个,有两位演讲者，3471 和5199，但却不容易找到他们，因为提供的是编号，
而`Schedule.speakers`列表没有使用编号建立索引。此外，每条事件记录中都有`venue_serial` 字段，存储的值也是编号，但是如果想找到对应的记录，那就要线性搜索`Schedule.venues`列表.接下来的任务是，调整数据结构，以便自动获取所链接的记录.

### 使用shelve模块调整OSCON数据源的结构

标准库中有个`shelve`(架子)模块，这名字听起来怪怪的，可是如果知道`pickle`(泡菜)是Python 对象序列化格式的名字，还是在那个格式与对象之间相互转换的某个模块的名字，就会觉得以`shelve` 命名是合理的。泡菜坛子摆放在架子上，因此shelve模块提供了pickle存储方式.

`shelve.open` 高阶函数返回一个shelve.Shelf 实例，这是简单的键值对象数据库，背后由`dbm`模块支持，具有下述特点:

+ shelve.Shelf 是abc.MutableMapping 的子类，因此提供了处理映射类型的重要方法。
+ 此外，shelve.Shelf 类还提供了几个管理I/O 的方法，如sync 和close；它也是一个上
下文管理器。
+ 只要把新值赋予键，就会保存键和值。
+ 键必须是字符串。
+ 值必须是pickle 模块能处理的对象。

shelve 模块为识别OSCON 的日程数据提供了一种简单有效的方式.我们将从JSON 文件中读取所有记录，将其存在一个shelve.Shelf 对象中，键由记录类型和编号组成（例如，`event.33950` 或`speaker.3471`），而值是我们即将定义的Record 类的实例.


In [25]:
import warnings

DB_NAME = 'schedule1_db'
CONFERENCE = 'conference.115'
class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs) 
def load_db(db):
    raw_data = load()
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = Record(**record)

In [26]:
import shelve

In [27]:
db = shelve.open(DB_NAME) 

In [28]:
if CONFERENCE not in db:
    load_db(db)

In [29]:
speaker = db['speaker.3471']

In [30]:
type(speaker)

__main__.Record

In [31]:
speaker.name, speaker.twitter

('Anna Martelli Ravenscroft', 'annaraven')

In [32]:
db.close()

`Record.__init__`方法展示了一个流行的Python 技巧。我们知道，对象的`__dict__` 属性中存储着对象的属性——前提是类中没有声明`__slots__` 属性.因此，更新实例的`__dict__`属性，把值设为一个映射，能快速地在那个实例中创建一堆属性.


示例中定义的`Record`类太简单了，因此你可能会问，为什么之前没用，而是使用更复杂的`FrozenJSON`类。原因有两个:

+ 第一，`FrozenJSON`类要递归转换嵌套的映射和列表；而Record 类不需要这么做，因为转换好的数据集中没有嵌套的映射和列表，记录中只有字符串、整数、字符串列表和整数列表.
+ 第二，FrozenJSON 类要访问内嵌的`__data `属性（值是字典，用于调用`keys`等方法），而现在我们也不需要这么做

像上面那样调整日程数据集之后，我们可以扩展`Record`类，让它提供一个有用的服务--自动获取`event`记录引用的`venue` 和`speaker`记录。这与`Django ORM` 访问`models.ForeignKey` 字段时所做的事类似--得到的不是键，而是链接的模型对象.

## 动态绑定方法

python中方法只是可以调用的属性,因此方法也是可以动态绑定的.尤其实例方法的动态绑定尤其实用.

### 动态绑定实例方法

动态绑定实例方法需要借助`types.MethodType`

In [33]:
from types import MethodType
class Student(object):
    age = 10
def set_age(self, age): # 定义一个函数作为实例方法
    self.age = age
s = Student()
s.set_age = MethodType(set_age, s) # 给实例绑定一个方法

In [34]:
s.age

10

In [35]:
s.set_age(12)
s.age

12

### 动态绑定类方法

动态绑定类方法与前面类似,只是`MethodType`的第一个参数改成了类名

In [36]:
def set_score(clz, score):#定义一个函数作为类的方法
     clz.score = score

Student.set_score = MethodType(set_score, Student)

In [37]:
Student.set_score(30)

In [38]:
Student.score

30

In [39]:
s.score

30

### 动态绑定静态方法

动态绑定静态方法更加简单了,只要直接在类名后面像添加元素一样添加即可

In [40]:
def echo(score):#定义一个函数作为类的方法
    return score
Student.echo = echo

In [41]:
Student.echo(123)

123

# 特性

除了属性之外，我们还可以创建特性(property)，在不改变类接口的前提下，使用存取方法(即读值方法和设值方法)修改数据属性.这与统一访问原则相符--不管服务是由存储还是计算实现的，一个模块提供的所有服务都应该通过统一的方式使用.


`property`是一个用于类中方法的装饰器,用于将方法属性转换为特性,如果要设定特性的增删改查能力,则可以使用`<property>.setter,<property>.deleter`定义.

```python
class Event(DbRecord): 
    @property
    def venue(self):
    '''The Event attribute'''
        return self.__venue
        
    @venue.setter
    def venue(self,value):
        self.__venue = value
        
    @venue.deleter
    def venue(self,value):
        del self.__venue     
    
```

虽然内置的`property`经常用作装饰器，但它其实是一个类。在Python 中，函数和类通常可以互换，因为二者都是可调用的对象，而且没有实例化对象的`new`运算符，所以调用构造方法与调用工厂函数没有区别。此外，只要能返回新的可调用对象，代替被装饰的函数，二者都可以用作装饰器.

`property`构造方法的完整签名如下：

`property(fget=None, fset=None, fdel=None, doc=None)`

所有参数都是可选的，如果没有把函数传给某个参数，那么得到的特性对象就不允许执行相应的操作.

某些情况下，这种经典形式比装饰器句法好.但是，在方法众多的类定义体中使用装饰器的话，一眼就能看出哪些是读值方法，哪些是设值方法，而不用按照惯例，在方法名的前面加上`get` 和`set`.类中的特性能影响实例属性的寻找方式，而一开始这种方式可能会让人觉得意外.

特性都是类属性，但是特性管理的其实是实例属性的存取.如果实例和所属的类有同名数据属性，那么实例属性会覆盖（或称遮盖）类属性——至少通过那个实例读取属性时是这样.


### 实例属性遮盖类的数据属性

In [42]:
class Class:
    data = 'the class data attr'
    @property
    def prop(self):
        return 'the prop value'

In [43]:
obj = Class()

In [44]:
vars(obj)

{}

In [45]:
obj.data

'the class data attr'

In [46]:
obj.data = 'bar'

In [47]:
vars(obj)

{'data': 'bar'}

In [48]:
obj.data

'bar'

In [49]:
Class.data

'the class data attr'

### 实例属性不会遮盖类特性

In [50]:
Class.prop

In [51]:
obj.prop

'the prop value'

In [52]:
obj.prop = 'foo'

AttributeError: can't set attribute

In [53]:
obj.__dict__['prop'] = 'foo'

In [54]:
vars(obj)

{'data': 'bar', 'prop': 'foo'}

In [55]:
obj.prop

'the prop value'

In [56]:
Class.prop

In [57]:
obj.prop

'the prop value'

### 新添的类特性遮盖现有的实例属性

In [58]:
obj.data

'bar'

In [59]:
Class.data

'the class data attr'

In [60]:
Class.data = property(lambda self: 'the "data" prop value')

In [61]:
obj.data

'the "data" prop value'

In [62]:
del Class.data

In [63]:
obj.data

'bar'

### 特性的文档

控制台中的`help()`函数或IDE 等工具需要显示特性的文档时，会从特性的`__doc__` 属性中
提取信息。

+ 如果使用经典调用句法，为`property`对象设置文档字符串的方法是传入`doc`参数：
    
    `weight = property(get_weight, set_weight, doc='weight in kilograms')`

+使用装饰器创建`property`对象时，读值方法（有`@property`装饰器的方法）的文档字符串作
为一个整体，变成特性的文档.

## 使用特性获取链接的记录

下图是用到的几个类
![](source/record.PNG)

+ Record
    
    `__init__` 方法与schedule1.py 脚本（见示例19-9）中的一样；为了辅助测试，增加了__eq__ 方法。

+ DbRecord

    Record 类的子类，添加了`__db` 类属性，用于设置和获取`__db` 属性的`set_db` 和`get_db`静态方法，用于从数据库中获取记录的fetch 类方法，以及辅助调试和测试的`__repr__`实例方法。

+ Event

    `DbRecord`类的子类，添加了用于获取所链接记录的`venue` 和`speakers` 属性，以及特殊的`__repr__` 方法。

In [64]:
import inspect
DB_NAME = 'schedule2_db' 
CONFERENCE = 'conference.115'
class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
    def __eq__(self, other): 
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented
        
class MissingDatabaseError(RuntimeError):
    """需要数据库但没有指定数据库时抛出。"""
    pass

class DbRecord(Record):
    __db = None
    @staticmethod 
    def set_db(db):
        DbRecord.__db = db
    @staticmethod
    def get_db():
        return DbRecord.__db
    @classmethod
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident]
        except TypeError:
            if db is None:
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else: 
                raise
                    
    def __repr__(self):
        if hasattr(self, 'serial'):
            cls_name = self.__class__.__name__
            return '<{} serial={!r}>'.format(cls_name, self.serial)
        else:
            return super().__repr__()
        
class Event(DbRecord): 
    @property
    def venue(self):
        key = 'venue.{}'.format(self.venue_serial)
        return self.__class__.fetch(key) 
    @property
    def speakers(self):
        if not hasattr(self, '_speaker_objs'): 
            spkr_serials = self.__dict__['speakers'] 
            fetch = self.__class__.fetch 
            self._speaker_objs = [fetch('speaker.{}'.format(key)) 
                                      for key in spkr_serials] 
        return self._speaker_objs 
    def __repr__(self):
        if hasattr(self, 'name'): 
            cls_name = self.__class__.__name__
            return '<{} {!r}>'.format(cls_name, self.name)
        else:
            return super().__repr__()
        
def load_db(db):
    raw_data = load()
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1] 
        cls_name = record_type.capitalize() 
        cls = globals().get(cls_name, DbRecord) 
        if inspect.isclass(cls) and issubclass(cls, DbRecord): 
            factory = cls 
        else:
            factory = DbRecord 
        for record in rec_list: 
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = factory(**record)

In [65]:
import shelve

In [66]:
db = shelve.open(DB_NAME) 

In [67]:
if CONFERENCE not in db:
    load_db(db)

In [68]:
DbRecord.set_db(db)

In [69]:
event = DbRecord.fetch('event.33950')
event

<Event 'There *Will* Be Bugs'>

In [70]:
event.venue

<DbRecord serial='venue.1449'>

In [71]:
event.venue.name

'Portland 251'

In [72]:
for spkr in event.speakers:
    print('{0.serial}: {0.name}'.format(spkr))

speaker.3471: Anna Martelli Ravenscroft
speaker.5199: Alex Martelli


In [73]:
db.close()

## 使用特性验证属性

目前，我们只介绍了如何使用`@property`装饰器实现只读特性。本节要创建一个可读写的特性

### `LineItem`类第1版：表示订单中商品的类

假设有个销售散装有机食物的电商应用，客户可以按重量订购坚果、干果或杂粮.在这个系统中，每个订单中都有一系列商品，而每个商品都可以使用.

In [74]:
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [75]:
raisins = LineItem('Golden raisins', 10, 6.95)

In [76]:
raisins.subtotal()

69.5

In [77]:
raisins.weight = -20

In [78]:
raisins.subtotal()

-139.0

这个类没法限制参数.比如作为一个商品订单,它的值可以是负的.

### `LineItem`类第2版：能验证值的特性

In [79]:
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price
    @property 
    def weight(self):
        return self.__weight 
    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weight = value 
        else:
            raise ValueError('value must be > 0')

## 特性工厂函数

我们的`weight` 和`price`有相似的特点,都不能为负.如果一个类有很多这样的特性,那一个一个写特性会很麻烦,因此可以使用特性工厂函数来产生一样特点的特性.


我们将定义一个名为`quantity`的特性工厂函数，取这个名字是因为，在这个应用中要管理的属性表示不能为负数或零的量.下例是`LineItem`类的简洁版，用到了`quantity`特性的两个实例：

+ 一个用于管理`weight`属性，
+ 另一个用于管理`price`属性。


In [80]:
def quantity(storage_name):
    def qty_getter(instance):
        return instance.__dict__[storage_name]
    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')
    return property(qty_getter, qty_setter)

class LineItem:
    weight = quantity('weight')
    price = quantity('price')
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [81]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)

In [82]:
nutmeg.weight, nutmeg.price

(8, 13.95)

In [83]:
sorted(vars(nutmeg).items())

[('description', 'Moluccan nutmeg'), ('price', 13.95), ('weight', 8)]

工厂函数构建的特性利用了特性覆盖实例属性的行为，因此对`self.weight` 或`nutmeg.weight` 的每个引用都由特性函数处理，只有直接存取`__dict__`属性才能跳过特性的处理逻辑.

在真实的系统中，分散在多个类中的多个字段可能要做同样的验证，此时最好把`quantity`工厂函数放在实用工具模块中，以便重复使用。最终可能要重构那个简单的工厂函数，改成更易扩展的描述符类，然后使用专门的子类执行不同的验证.

# 属性描述符

描述符是对多个属性运用相同存取逻辑的一种方式,ORM 中的字段类型是往往使用描述符，把数据库记录中字段里的数据与Python对象的属性对应起来.

描述符是实现了特定协议的类， 这个协议包括`__get__`、`__set__` 和`__delete__` 方法.

`property`类实现了完整的描述符协议.通常，可以只实现部分协议.其实，我们在真实的代码中见到的大多数描述符只实现了`__get__` 和`__set__`方法，还有很多只实现了其中的一个.描述符是Python的独有特征,不仅在应用层中使用，在语言的基础设施中也有用到.除了特性之外，使用描述符的Python功能还有方法及`classmethod`和`staticmethod`装饰器。理解描述符是精通Python的关键.

## LineItem类第3版：一个简单的描述符

实现了`__get__`、`__set__` 或`__delete__ `方法的类是描述符。描述符的用法是，创建一个实
例，作为另一个类的类属性.

我们将定义一个`Quantity`描述符用来代替特性工厂函数，`LineItem` 类会用到两个`Quantity`实例：

+ 一个用于管理weight 属性，
+ 另一个用于管理price 属性。

![](source/des.PNG)


从现在开始，我会使用下述定义:

+ 描述符类

    实现描述符协议的类。在上图中，是Quantity 类。

+ 托管类

    把描述符实例声明为类属性的类——上图中的LineItem 类。

+ 描述符实例

    描述符类的各个实例，声明为托管类的类属性。在上图中，各个描述符实例使用箭头和带下划线的名称表示（在UML 中，下划线表示类属性）.与黑色菱形接触的LineItem 类包含描述符实例.
    
+ 托管实例

    托管类的实例.在这个示例中，LineItem 实例是托管实例

+ 储存属性

    托管实例中存储自身托管属性的属性。在上图中，LineItem 实例的`weight` 和`price`属性是储存属性.这种属性与描述符属性不同，描述符属性都是类属性.
    
+ 托管属性

    托管类中由描述符实例处理的公开属性，值存储在储存属性中.也就是说，描述符实例和储存属性为托管属性建立了基础.

In [84]:
class Quantity:
    def __init__(self, storage_name):
        self.storage_name = storage_name
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            raise ValueError('value must be > 0')

各个托管属性的名称与储存属性一样，而且读值方法不需要特殊的逻辑，所以Quantity 类不需要定义`__get__`方法.

编写`__set__`方法时，要记住`self` 和`instance` 参数的意思：

+ self 是描述符实例，
+ instance 是托管实例

管理实例属性的描述符应该把值存储在托管实例中。因此，Python 才为描述符中的那个方法提供了`instance`参数.

In [85]:
class LineItem:
    weight = Quantity('weight')
    price = Quantity('price')
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [86]:
truffle = LineItem('White truffle', 100, 0)

ValueError: value must be > 0

上面的方式还是不够简洁,我们不得不在申明LineItem时为每个属性指定`Quantity()`的参数--属性的名称.

可问题是，赋值语句右手边的表达式先执行，而此时变量还不存在.

`Quantity()` 表达式计算的结果是创建描述符实例，而此时Quantity类中的代码无法猜出要把描述符绑定给哪个变量(例如weight 或price).

因此，上例必须明确指明各个`Quantity`实例的名称.这样不仅麻烦，还很危险：
如果程序员直接复制粘贴代码而忘了编辑名称，比如写成`price = Quantity('weight')`，那
么程序的行为会大错特错，设置`price `的值时会覆盖`weight` 的值.

## LineItem类第4版：自动获取储存属性的名称

为了避免在描述符声明语句中重复输入属性名，我们将为每个Quantity 实例的`storage_name` 属性生成一个独一无二的字符串。下图是更新后的Quantity 和LineItem 类的UML类图.

![](source/lineitem4.PNG)

为了生成storage_name， 我们以`'_Quantity#'` 为前缀，然后在后面拼接一个整数：

`Quantity.__counter` 类属性的当前值，每次把一个新的Quantity 描述符实例依附到类上，都会递增这个值。在前缀中使用井号能避免`storage_name`与用户使用点号创建的属性冲突，因为`nutmeg._Quantity#0` 是无效的Python句法.但是，内置的`getattr` 和`setattr`函数可以使用这种“无效的”标识符获取和设置属性，此外也可以直接处理实例属性`__dict__`

In [87]:
class Quantity:
    __counter = 0
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')

In [88]:
class LineItem:
    weight = Quantity()
    price = Quantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [89]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)

In [90]:
coconuts.weight, coconuts.price

(20, 17.95)

In [91]:
getattr(coconuts, '_Quantity#0'), getattr(coconuts, '_Quantity#1')

(20, 17.95)

## LineItem类第5版：一种新型描述符

我们虚构的有机食物网店遇到一个问题：不知怎么回事儿，有个商品的描述信息为空，导致无法下订单.为了避免出现这个问题，我们要再创建一个描述符，NonBlank.在设计NonBlank的过程中，我们发现，它与Quantity描述符很像，只是验证逻辑不同.

回想Quantity的功能，我们注意到它做了两件不同的事：

+ 管理托管实例中的储存属性
+ 验证用于设置那两个属性的值

由此可知，我们可以重构，并创建两个基类

+ AutoStorage

    自动管理储存属性的描述符类
    
+ Validated

    扩展AutoStorage类的抽象子类，覆盖`__set__` 方法，调用必须由子类实现的`validate`方法

我们重写`Quantity`类，并实现`NonBlank`，让它继承`Validated`类，只编写`validate`方法.类之间的关系见图.

![](source/lineitem5.PNG)

`Validated`、`Quantity`和`NonBlank` 三个类之间的关系体现了模板方法设计模式.具体而言，`Validated.__set__` 方法正是`Gamma`等四人所描述的模板方法的例证：

    一个模板方法用一些抽象的操作定义一个算法，而子类将重定义这些操作以提供具体的行为.


In [92]:
import abc
class AutoStorage:
    __counter = 0
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)

In [93]:
class Validated(abc.ABC, AutoStorage): 
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
    @abc.abstractmethod
    def validate(self, instance, value):
        """return validated value or raise ValueError"""
        pass

In [94]:
class Quantity(Validated): 
    """a number greater than zero"""
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value

In [95]:
class NonBlank(Validated):
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value

In [96]:
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [97]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)

In [98]:
coconuts.weight, coconuts.price

(20, 17.95)

In [99]:
raisins = LineItem('Golden raisins', 10, 6.95)
dir(raisins)[:3]

['_NonBlank#0', '_Quantity#0', '_Quantity#1']

In [100]:
LineItem.description.storage_name

'_NonBlank#0'

## 覆盖型与非覆盖型描述符

Python存取属性的方式特别不对等。通过实例读取属性时，通常返回的是实例中定义的属性；但是，如果实例中没有指定的属性，那么会获取类属性.而为实例中的属性赋值时，通常会在实例中创建属性，根本不影响类.这种不对等的处理方式对描述符也有影响.其实，根据是否定义`__set__`方法，描述符可分为两大类.其中覆盖型又可以分为2小类.

+ 覆盖型

    定义`__set__`,描述符的`__set__`方法使用托管实例中的同名属性覆盖（即插手接管）了要设置的属性,这种类型描述符的典型用途是管理数据属性
    
    + 没有`__get__`方法的覆盖型描述符
    
        通常，覆盖型描述符既会实现`__set__` 方法，也会实现`__get__`方法，不过也可以只实现`__set__` 方法.此时，只有写操作由描述符处理。通过实例读取描述符会返回描述符对象本身，因为没有处理读操作的`__get__` 方法。如果直接通过实例的`__dict__`属性创建同名实例属性，以后再设置那个属性时，仍会由`__set__` 方法插手接管，但是读取那个属性的话，就会直接从实例中返回新赋予的值，而不会返回描述符对象。也就是说，实例属性会遮盖描述符，不过只有读操作是如此
    
    
+ 非覆盖型

    没有实现`__set__`方法的描述符是非覆盖型描述符.如果设置了同名的实例属性，描述符会被遮盖，致使描述符无法处理那个实例的那个属性.方法是以非覆盖型描述符实现的

我们通过下面的例子观察这两类描述符的行为差异

In [101]:
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]
def display(obj):
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return '<{} object>'.format(cls_name(obj))
def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))

In [102]:
class Overriding:
    """覆盖型描述符
    也称数据描述符或强制描述符"""
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

In [103]:
class OverridingNoGet:
    """没有`__get__`方法的覆盖型描述符"""
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

In [104]:
class NonOverriding:
    """也称非数据描述符或遮盖型描述符"""
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)

In [105]:
class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()
    def spam(self):
        print('-> Managed.spam({})'.format(display(self)))

### 覆盖型描述符的行为

上面的例子都是覆盖型描述符

In [106]:
obj = Managed()

In [107]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [108]:
Managed.over

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [109]:
obj.over = 7

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [110]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [111]:
obj.__dict__['over'] = 8

In [112]:
vars(obj)

{'over': 8}

In [113]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


### 没有`__get__`的覆盖型描述符的行为

只有写操作由描述符处理。通过实例读取描述符会返回描述符对象本身，

In [114]:
obj.over_no_get

In [115]:
Managed.over_no_get

In [116]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [117]:
obj.over_no_get

In [118]:
obj.__dict__['over_no_get'] = 9

In [119]:
obj.over_no_get

9

In [120]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [121]:
obj.over_no_get

9

### 非覆盖型描述符的行为

如果设置了同名的实例属性，描述符会被遮盖，致使描述符无法处理那个实例的那个属性

In [122]:
obj = Managed()

In [123]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [124]:
obj.non_over = 7

In [125]:
obj.non_over

7

In [126]:
Managed.non_over

-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)


In [127]:
del obj.non_over

In [128]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


## 在类中覆盖描述符

依附在类上的描述符无法控制为类属性赋值的操作。其实，这意味着为类属性赋值能覆盖描述符属性.这是一种猴子补丁技术，不过在下例中，我们把描述符替换成了整数，这其实会导致依赖描述符的类不能正确地执行操作.

In [129]:
obj = Managed()

In [130]:
Managed.over = 1
Managed.over_no_get = 2
Managed.non_over = 3
obj.over, obj.over_no_get, obj.non_over

(1, 2, 3)

读类属性的操作可以由依附在托管类上定义有`__get__` 方法的描述符处理，但是写类属性的操作不会由依附在托管类上定义有`__set__`方法的描述符处理.

若想控制设置类属性的操作，要把描述符依附在类的类上，即依附在元类上.默认情况下，对用户定义的类来说，其元类是`type`，而我们不能为`type` 添加属性,但我们可以自定义元类.

## 描述符协议增强(3.6)

上面的`LineItem`有个缺陷--就是初始化的时候都明确让属性的值绑定在Integer上的name属性上，而无法获知所有者类的属性名。如果使用自定义内部名字,又会难以调试.使用在[PEP487](https://www.python.org/dev/peps/pep-0487/)上提供的可选的`__set_name__()`可以获得这个属性名字，并且可以自定义这部分内容：

In [131]:
class AutoStorage:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.name)
    def __set__(self, instance, value):
        setattr(instance, self.name, value) 
    def __set_name__(self, owner, name):
        cls = self.__class__
        prefix = cls.__name__
        index = name
        self.name = '_{}#{}'.format(prefix, index)
        
class Validated(abc.ABC, AutoStorage): 
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
    @abc.abstractmethod
    def validate(self, instance, value):
        """return validated value or raise ValueError"""
        pass
class Quantity(Validated): 
    """a number greater than zero"""
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value
    
class NonBlank(Validated):
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value

In [132]:
class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [133]:
raisins = LineItem('Golden raisins', 10, 6.95)
dir(raisins)[:3]

['_NonBlank#description', '_Quantity#price', '_Quantity#weight']

In [134]:
LineItem.description.name

'_NonBlank#description'

## 方法是描述符

在类中定义的函数属于绑定方法（bound method），因为用户定义的函数都有`__get__`方法，所以依附到类上时，就相当于描述符.函数没有实现`__set__`方法，因此是非覆盖型描述符.

与描述符一样，通过托管类访问时，函数的`__get__`方法会返回自身的引用。但是，通过实例访问时，函数的`__get__`方法返回的是绑定方法对象：一种可调用的对象，里面包装着函数，并把托管实例（例如`obj`）绑定给函数的第一个参数（即`self`），这与`functools.partial`函数的行为一致

## 描述符用法建议

下面根据刚刚论述的描述符特征给出一些实用的结论:

+ 使用特性以保持简单

    内置的property 类创建的其实是覆盖型描述符，`__set__ `方法和`__get__` 方法都实现了，即便不定义设值方法也是如此。特性的`__set__ `方法默认抛出`AttributeError:can't set attribute`，因此创建只读属性最简单的方式是使用特性，这能避免下一条所述的问题.

+ 只读描述符必须有`__set__`方法

    如果使用描述符类实现只读属性， 要记住，`__get__` 和`__set__` 两个方法必须都定义，否则，实例的同名属性会遮盖描述符。只读属性的`__set__`方法只需抛出`AttributeError` 异常，并提供合适的错误消息.
    
+ 用于验证的描述符可以只有`__set__`方法

    对仅用于验证的描述符来说，`__set__` 方法应该检查value参数获得的值，如果有效，使用描述符实例的名称为键，直接在实例的`__dict__`属性中设置。这样，从实例中读取同名属性的速度很快，因为不用经过`__get__`方法处理.
    
+ 仅有`__get__`方法的描述符可以实现高效缓存

    如果只编写了`__get__`方法，那么创建的是非覆盖型描述符。这种描述符可用于执行某些耗费资源的计算，然后为实例设置同名属性，缓存结果.同名实例属性会遮盖描述符，因此后续访问会直接从实例的`__dict__`属性中获取值，而不会再触发描述符的`__get__`方法。

+ 非特殊的方法可以被实例属性遮盖

    由于函数和方法只实现了`__get__` 方法，它们不会处理同名实例属性的赋值操作.因此，像`my_obj.the_method = 7` 这样简单赋值之后，后续通过该实例访问`the_method`得到的是数字7——但是不影响类或其他实例.然而，特殊方法不受这个问题的影响.解释器只会在类中寻找特殊的方法，也就是说，`repr(x)` 执行的其实是`x.__class__.__repr__(x)`，因此x的`__repr__` 属性对`repr(x)`方法调用没有影响。出于同样的原因，实例的`__getattr__`属性不会破坏常规的属性访问规则.